In [1]:
import torch
from pcdet.ops.iou3d_nms import iou3d_nms_cuda
torch.ops.load_library(".\iou3d_nms_cuda.cp310-win_amd64.pyd")

In [2]:
model = torch.jit.load("./tools/torch_scripts/point_pillar_model.pt")
model.cuda()
model.eval()

RecursiveScriptModule(
  original_name=PointPillar
  (vfe): RecursiveScriptModule(
    original_name=PillarVFE
    (pfn_layers): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=PFNLayer
        (linear): RecursiveScriptModule(original_name=Linear)
        (norm): RecursiveScriptModule(original_name=BatchNorm1d)
      )
    )
  )
  (map_to_bev_module): RecursiveScriptModule(original_name=PointPillarScatter)
  (backbone_2d): RecursiveScriptModule(
    original_name=BaseBEVBackbone
    (blocks): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=ZeroPad2d)
        (1): RecursiveScriptModule(original_name=Conv2d)
        (2): RecursiveScriptModule(original_name=BatchNorm2d)
        (3): RecursiveScriptModule(original_name=ReLU)
        (4): RecursiveScriptModule(original_name=Conv2d)
        (5): RecursiveScript

In [3]:
def nms_gpu(boxes, scores, thresh, pre_maxsize=None, **kwargs):
    """
    :param boxes: (N, 7) [x, y, z, dx, dy, dz, heading]
    :param scores: (N)
    :param thresh:
    :return:
    """
    assert boxes.shape[1] == 7
    order = scores.sort(0, descending=True)[1]
    if pre_maxsize is not None:
        order = order[:pre_maxsize]

    boxes = boxes[order].contiguous()
    # keep = torch.IntTensor(boxes.size(0))
    keep = torch.zeros(boxes.size(0)).int()

    num_out = iou3d_nms_cuda.nms_gpu(boxes, keep, thresh)

    return order[keep[:num_out].long().cuda()].contiguous(), None

def class_agnostic_nms(box_scores, box_preds, score_thresh=None):
    src_box_scores = box_scores
    if score_thresh is not None:
        scores_mask = (box_scores >= score_thresh)
        box_scores = box_scores[scores_mask]
        box_preds = box_preds[scores_mask]

    selected = []
    if box_scores.shape[0] > 0:
        box_scores_nms, indices = torch.topk(box_scores, k=min(4096, box_scores.shape[0]))
        boxes_for_nms = box_preds[indices]
        keep_idx, selected_scores = nms_gpu(
                boxes_for_nms[:, 0:7], box_scores_nms, score_thresh, 4096
        )
        selected = indices[keep_idx[:500]]

    if score_thresh is not None:
        original_idxs = scores_mask.nonzero().view(-1)
        selected = original_idxs[selected]
    return selected, src_box_scores[selected]

In [4]:
test_1 = torch.load("./test_dataset_000000.pt")
test_1


{'points': tensor([[ 0.0000e+00,  1.8324e+01,  4.9000e-02,  8.2900e-01,  0.0000e+00],
         [ 0.0000e+00,  1.8344e+01,  1.0600e-01,  8.2900e-01,  0.0000e+00],
         [ 0.0000e+00,  5.1299e+01,  5.0500e-01,  1.9440e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  3.7180e+00, -1.4060e+00, -1.7370e+00,  3.4000e-01],
         [ 0.0000e+00,  3.7140e+00, -1.3910e+00, -1.7330e+00,  4.1000e-01],
         [ 0.0000e+00,  3.9670e+00, -1.4740e+00, -1.8570e+00,  0.0000e+00]],
        device='cuda:0'),
 'frame_id': tensor([0], device='cuda:0', dtype=torch.int32),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'voxels': tensor([[[ 1.8324e+01,  4.9000e-02,  8.2900e-01,  0.0000e+00],
          [ 1.8344e+01,  1.0600e-01,  8.2900e-01,  0.0000e+00],
          [ 1.8302e+01,  6.0000e-03,  7.1200e-01,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  

In [5]:
res_dict, _ = model(test_1)
res_dict


{'pred_boxes': tensor([[  0.7985, -39.9357,  -0.5309,  ...,   1.5980,   1.6815,   5.8274],
         [  0.6357, -40.8358,  -0.4494,  ...,   1.6770,   1.6024,   5.3345],
         [  0.0894, -39.6476,  -0.1480,  ...,   0.5903,   1.8044,   3.9129],
         ...,
         [ 69.1877,  39.7734,   0.2010,  ...,   0.5878,   1.8058,   4.8621],
         [ 69.3392,  39.7340,   0.2427,  ...,   0.4276,   1.7731,   3.5186],
         [ 69.1991,  39.9978,   0.3405,  ...,   0.4200,   1.9362,   4.5756]],
        device='cuda:0', grad_fn=<SelectBackward0>),
 'pred_scores': tensor([0.0046, 0.0018, 0.0003,  ..., 0.0009, 0.0007, 0.0008], device='cuda:0',
        grad_fn=<MaxBackward0>),
 'pred_labels': tensor([1, 1, 2,  ..., 2, 3, 3], device='cuda:0')}

In [7]:
cls_preds = res_dict["pred_scores"]
box_preds = res_dict["pred_boxes"]
label_preds = res_dict["pred_labels"]

selected, selected_scores = class_agnostic_nms(
                    box_scores=cls_preds, box_preds=box_preds,
                    score_thresh=0.01
                )

final_scores = selected_scores
final_labels = label_preds[selected]
final_boxes = box_preds[selected]

